In [6]:
!pip install openpyxl

  Using cached openpyxl-3.0.7-py2.py3-none-any.whl (243 kB)


In [5]:
!pip install progressbar

  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12075 sha256=cf6514b3af9d8bf260d8657a602c0a319ec50026e472ff914cd1784f92e60a19
  Stored in directory: c:\users\j\appdata\local\pip\cache\wheels\f0\fd\1f\3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar


In [7]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup

import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm

###### 날짜 저장 ##########
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [8]:
####### 언론사별 본문 위치 태그 파싱 함수 ###########
print('본문 크롤링에 필요한 함수를 로딩하고 있습니다...\n' + '-' * 100)



def crawling_main_text(url):

    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('div', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
        
    # 매일경제(미라클), req.encoding = None 설정 필요
    elif 'mirakle.mk' in url:
        text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
    elif 'mk.co' in url:
        text = soup.find('div', {'class' : 'art_txt'}).text
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 머니투데이
    elif 'news.mt' in url:
        text = soup.find('div', {'class' : 'view_text'}).text
        
    # 그 외
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')
    
    
press_nm = '머니투데이'

print('검색할 언론사 : {}'.format(press_nm))


############### 브라우저를 켜고 검색 키워드 입력 ####################
query = input('검색할 키워드  : ')
news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))

print('\n' + '=' * 100 + '\n')

print('브라우저를 실행시킵니다(자동 제어)\n')
chrome_path = '.\\webdriver\\chromedriver.exe'
browser = webdriver.Chrome(chrome_path)

news_url = 'https://search.naver.com/search.naver?where=news&query={}'.format(query)
browser.get(news_url)
time.sleep(sleep_sec)



######### 언론사 선택 및 confirm #####################
print('설정한 언론사를 선택합니다.\n')

search_opn_btn = browser.find_element_by_xpath('//a[@class="btn_option _search_option_open_btn"]')
search_opn_btn.click()
time.sleep(sleep_sec)

bx_press = browser.find_element_by_xpath('//div[@role="listbox" and @class="api_group_option_sort _search_option_detail_wrap"]//li[@class="bx press"]')

# 기준 두번 째(언론사 분류순) 클릭하고 오픈하기
press_tablist = bx_press.find_elements_by_xpath('.//div[@role="tablist" and @class="option"]/a')
press_tablist[1].click()
time.sleep(sleep_sec)

# 첫 번째 것(언론사 분류선택)
bx_group = bx_press.find_elements_by_xpath('.//div[@class="api_select_option type_group _category_select_layer"]/div[@class="select_wrap _root"]')[0]

press_kind_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[0]
press_kind_btn_list = press_kind_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')


for press_kind_btn in press_kind_btn_list:
    
    # 언론사 종류를 순차적으로 클릭(좌측)
    press_kind_btn.click()
    time.sleep(sleep_sec)
    
    # 언론사선택(우측)
    press_slct_bx = bx_group.find_elements_by_xpath('.//div[@class="group_select _list_root"]')[1]
    # 언론사 선택할 수 있는 클릭 버튼
    press_slct_btn_list = press_slct_bx.find_elements_by_xpath('.//ul[@role="tablist" and @class="lst_item _ul"]/li/a')
    # 언론사 이름들 추출
    press_slct_btn_list_nm = [psl.text for psl in press_slct_btn_list]
    
    # 언론사 이름 : 언론사 클릭 버튼 인 딕셔너리 생성
    press_slct_btn_dict = dict(zip(press_slct_btn_list_nm, press_slct_btn_list))
    
    # 원하는 언론사가 해당 이름 안에 있는 경우
    # 1) 클릭하고
    # 2) 더이상 언론사분류선택 탐색 중지
    if press_nm in press_slct_btn_dict.keys():
        print('<{}> 카테고리에서 <{}>를 찾았으므로 탐색을 종료합니다'.format(press_kind_btn.text, press_nm))
        
        press_slct_btn_dict[press_nm].click()
        time.sleep(sleep_sec)
        
        break
        
        
        
################ 뉴스 크롤링 ########################

print('\n크롤링을 시작합니다.')
# ####동적 제어로 페이지 넘어가며 크롤링
news_dict = {}
idx = 1
cur_page = 1

pbar = tqdm(total=news_num ,leave = True)
    
while idx < news_num:

    table = browser.find_element_by_xpath('//ul[@class="list_news"]')
    li_list = table.find_elements_by_xpath('./li[contains(@id, "sp_nws")]')
    area_list = [li.find_element_by_xpath('.//div[@class="news_area"]') for li in li_list]
    a_list = [area.find_element_by_xpath('.//a[@class="news_tit"]') for area in area_list]
 
    for n in a_list[:min(len(a_list), news_num-idx+1)]:
        n_url = n.get_attribute('href')
        news_dict[idx] = {'title' : n.get_attribute('title'), 
                          'url' : n_url,
                          'text' : crawling_main_text(n_url)}
        
        idx += 1
        pbar.update(1)
        
    if idx < news_num:
        cur_page +=1

        pages = browser.find_element_by_xpath('//div[@class="sc_page_inner"]')
        next_page_url = [p for p in pages.find_elements_by_xpath('.//a') if p.text == str(cur_page)][0].get_attribute('href')

        browser.get(next_page_url)
        time.sleep(sleep_sec)
    else:
        pbar.close()
        
        print('\n브라우저를 종료합니다.\n' + '=' * 100)
        time.sleep(0.7)
        browser.close()
        break
        
#### 데이터 전처리하기 ###################################################### 

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_본문_{}개_{}_{}.xlsx'.format(news_num, query, date)

news_df.to_excel(xlsx_file_name)

print('엑셀 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, xlsx_file_name))

os.startfile(folder_path)

print('=' * 100 + '\n결과물의 일부')
news_df

본문 크롤링에 필요한 함수를 로딩하고 있습니다...
----------------------------------------------------------------------------------------------------
검색할 언론사 : 머니투데이


검색할 키워드  :  삼성전자
수집 뉴스의 수(숫자만 입력) :  100




브라우저를 실행시킵니다(자동 제어)

설정한 언론사를 선택합니다.

<경제/IT> 카테고리에서 <머니투데이>를 찾았으므로 탐색을 종료합니다


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]


크롤링을 시작합니다.


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [08:21<00:00,  5.01s/it]



브라우저를 종료합니다.
데이터프레임 변환

엑셀 저장 완료 | 경로 : C:\Users\j\stock_predict\크롤링\네이버뉴스_본문_100개_삼성전자_2021-08-31_10시59분.xlsx

결과물의 일부


,title,url,text
1,"KH바텍, 삼성전자 폴더블폰 돌풍 수혜…목표가 ↑-한국투자증권",http://news.mt.co.kr/mtview.php?no=20210831085...,12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤럭시Z폴...
2,5G 표준 주도하는 삼성전자 …3GPP '최다 의장석' 확보,http://news.mt.co.kr/mtview.php?no=20210830094...,지난 8월16일부터 28일까지 개최된 3GPP 표준 회의에서 삼성리서치의 송재연 연...
3,'삼성전자 배당' 받은 보험사…상반기 영업이익 50%↑,http://news.mt.co.kr/mtview.php?no=20210827181...,삼성전자 특별배당 효과 등으로 국내 보험회사의 올 상반기 당기순이익이 지난해보다 5...
4,"스마트폰 왕좌 노리는 삼성, 폴더블·중저가 '쌍끌이 전략' 통할까",http://news.mt.co.kr/mtview.php?no=20210830163...,12일 서울 서초구 삼성 딜라이트를 찾은 고객들이 새로운 폴더블 스마트폰 갤럭시Z폴...
5,"삼성전자, 상승출발…Z폴더3·Z플립3 흥행효과?",http://news.mt.co.kr/mtview.php?no=20210830091...,"삼성전자 (74,500원 100 -0.1%)가 30일 유가증권시장에서 상승세로 출발..."
...,...,...,...
96,삼성의 폴더블 야심…'갤럭시Z폴드3·Z플립3' 베일 벗는다,http://news.mt.co.kr/mtview.php?no=20210810150...,갤럭시Z폴드3/사진=에반블래스 트위터삼성전자의 하반기 신작 폴더블폰이 베일을 벗는다...
97,삼성전자 수원사업장서 15명 '코로나 확진'…건물 폐쇄,http://news.mt.co.kr/mtview.php?no=20210803161...,"경기도 수원시 영통구 삼성전자 수원사업장 모습./사진=뉴스1삼성전자 (74,600원..."
98,"'D램값 하락' 전망에…삼성전자 ""하반기 수요 견조할 것""",http://news.mt.co.kr/mtview.php?no=20210813095...,삼성전자 평택캠퍼스 P2 라인 전경./사진=뉴스1(삼성전자 제공)PC용 D램 가격이...
99,"삼성전자-10%, SK하이닉스-16%…'패닉셀' 이유 3가지",http://news.mt.co.kr/mtview.php?no=20210813173...,[서울=뉴시스]조성우 기자 = 삼성전자가 최신 DDR5 D램 모듈의 성능을 극대화...
